<a href="https://colab.research.google.com/github/anderson-silva21/CheckersSolutionAG/blob/anderson-branch/CheckersEvolutionaryApproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Método:


1.Gere uma população inicial de indivíduos aleatoriamente.

2.Avalie a aptidão de cada indivíduo na população.

3.Repita quantas vezes quiser:

  a) Selecione indivíduos com uma boa pontuação de aptidão para reprodução.

  b) Deixe-os produzir descendentes.

  c) Mudar esses descendentes.

  d) Avaliar a aptidão de cada indivíduo na população.

  f) Deixe que os indivíduos com um score de aptidão ruim morram.
  
4.Escolha o indivíduo com a maior aptidão como solução.

--------------------------------------------------------------------------------

Quais são os indivíduos? R: Soluções potenciais para o problema.

Quantos indivíduos estão na primeira população e como eles são gerados? R: precisa inventar. Se não tem ideia de qual poderia ser a solução, escolha a população inicial o mais aleatória possível.

Qual pontuação de condicionamento físico? R: inventar isso também. Nossa solução deve ter a pontuação de aptidão mais alta.

Quantos pais produzem quantos filhos? E como exatamente? R: inventar isso também

Como mutar? R: Altere os descendentes um pouco probabilisticamente. Esta é uma boa maneira de explorar mais soluções possíveis em diferentes lugares e não seguir o caminho errado de forma determinística.

Quantos indivíduos morrem? R: Quantos você quiser. Talvez seja bom manter a população do mesmo tamanho ao longo dos degraus, caso contrário o tamanho da população pode explodir ou todos morrem em algum momento.

Repetir com que frequência? R: Quantas vezes você quiser. Apenas verifique algumas vezes, até ver que as soluções não melhoram mais.

--------------------------------------------------------------------------------

Necessary imports

In [96]:
import numpy as np
from abc import ABC, abstractmethod

Class to define genetic algorithm structure

In [97]:
class Individual(ABC):
    def __init__(self, val=None, init_params=None):
        if val is not None:
            self.val = val
        else:
            self.val = self.random_init(init_params)

    @abstractmethod
    def paridade(self, other, paridade_params):
        pass

    @abstractmethod
    def mutacao(self, mutacao_params):
        pass

    @abstractmethod
    def random_init(self, init_params):
        pass

Algoritmo genetico para o jogo de damas

In [98]:
class Checkers(Individual):
    def paridade(self, other, pair_params):
        return Checkers(pair_params['alpha'] * self.val + (1 - pair_params['alpha']) * other.val)

    def mutacao(self, mutate_params):
        for _ in range(mutate_params['rate']):
            i, j = np.random.choice(range(len(self.value)), 2, replace=False)
            self.value[i], self.value[j] = self.value[j], self.value[i]

    def random_init(self, init_params):
        return np.random.choice(range(init_params['n_Enemy_Pecas']), init_params['n_Enemy_Pecas'], replace=False)

Population

In [99]:
from numpy.lib.npyio import packbits
class Populacao:
    def __init__(self, size, fitness, individual_class, init_params):
        self.fitness = fitness
        self.individuals = [individual_class(init_params=init_params) for _ in range(size)]
        self.individuals.sort(key=lambda x: self.fitness(x))

    def substituicao(self, new_individuals):
        size = len(self.individuals)
        self.individuals.extend(new_individuals)
        self.individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = self.individuals[-size:]

    def get_parents(self, n_filhos):
        maes = self.individuals[-2 * n_filhos::2]
        pais = self.individuals[-2 * n_filhos + 1::2]

        return maes, pais

Evolution

In [100]:
class Evolucao:
    def __init__(self, pool_size, fitness, individual_class, n_filhos, pair_params, mutate_params, init_params):
        self.pair_params = pair_params
        self.mutate_params = mutate_params
        self.pool = Populacao(pool_size, fitness, individual_class, init_params)
        self.n_filhos = n_filhos

    def etapa(self):
        maes, pais = self.pool.get_parents(self.n_filhos)
        filhos = []

        for mae, pai in zip(maes, pais):
            filho = mae.paridade(pai, self.pair_params)
            filho.mutacao(self.mutate_params)
            filhos.append(filho)

        self.pool.substituicao(filhos)

main code

In [101]:
def checkers_fitness_creator(EnemyPecas):
    matrix = []
    for city in EnemyPecas:
        row = []
        for city_ in EnemyPecas:
            row.append(np.linalg.norm(city - city_))
        matrix.append(row)
    distances = np.array(matrix)

    def fitness(checkers):
        res = 0
        for i in range(len(checkers.value)):
            res += distances[checkers.value[i], checkers.value[(i + 1) % len(checkers.value)]]
        return -res

    return fitness

evo = Evolucao(
    pool_size=10, fitness=checkers_fitness_creator, individual_class=Checkers, n_filhos=7,
    pair_params={'alpha': 0.5},
    mutate_params={'rate': 0.25},
    init_params={'n_Enemy_Pecas': 12}
)
n_epocas = 100

for i in range(n_epocas):
    evo.etapa()

print(evo.pool.individuals[-1].val)

[3.53844253]
